In [223]:
! pip install psycopg2
! pip install pandas

In [224]:
import psycopg2
import pandas as pd
import re

In [225]:
host = 'materialserver-3163.j77.aws-ap-southeast-1.cockroachlabs.cloud'
port = '26257'
dbname = 'material'
user = 'vendor'
password = 'k_wEbjoSUNNXFzdjxdzxTg'

# Connecting to the database
conn = psycopg2.connect(
    host=host,
    port=port,
    dbname=dbname,
    user=user,
    password=password,
    sslmode='verify-full',
    options='--cluster=materialserver-3163'  # Cluster identifier
)

## Product Processing

In [226]:
# Querying the database for product table
query = """SELECT * FROM product"""
product_df = pd.read_sql(query, conn)
product_df

/tmp/ipykernel_33387/4243177605.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  product_df = pd.read_sql(query, conn)


,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,...,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,,1000,750,0,0,...,,,,,,2008-04-30 00:00:00,,,{694215B7-08F7-4C0D-ACB1-D734BA44C0C8},2014-02-08 10:01:36.827000000
1,2,Bearing Ball,BA-8327,False,False,,1000,750,0,0,...,,,,,,2008-04-30 00:00:00,,,{58AE3C20-4F3A-4749-A7D4-D568806CC537},2014-02-08 10:01:36.827000000
2,3,BB Ball Bearing,BE-2349,True,False,,800,600,0,0,...,,,,,,2008-04-30 00:00:00,,,{9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E},2014-02-08 10:01:36.827000000
3,4,Headset Ball Bearings,BE-2908,False,False,,800,600,0,0,...,,,,,,2008-04-30 00:00:00,,,{ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B},2014-02-08 10:01:36.827000000
4,316,Blade,BL-2036,True,False,,800,600,0,0,...,,,,,,2008-04-30 00:00:00,,,{E73E9750-603B-4131-89F5-3DD15ED5FF80},2014-02-08 10:01:36.827000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,995,ML Bottom Bracket,BB-8107,True,True,,500,375,44,101,...,,M,,5,96,2013-05-30 00:00:00,,,{71AB847F-D091-42D6-B735-7B0C2D82FC84},2014-02-08 10:01:36.827000000
500,996,HL Bottom Bracket,BB-9108,True,True,,500,375,53,121,...,,H,,5,97,2013-05-30 00:00:00,,,{230C47C5-08B2-4CE3-B706-69C0BDD62965},2014-02-08 10:01:36.827000000
501,997,"Road-750 Black, 44",BK-R19B-44,True,True,Black,100,75,343,539,...,R,L,U,2,31,2013-05-30 00:00:00,,,{44CE4802-409F-43AB-9B27-CA53421805BE},2014-02-08 10:01:36.827000000
502,998,"Road-750 Black, 48",BK-R19B-48,True,True,Black,100,75,343,539,...,R,L,U,2,31,2013-05-30 00:00:00,,,{3DE9A212-1D49-40B6-B10A-F564D981DBDE},2014-02-08 10:01:36.827000000


In [227]:
# Filtering the product table for the required pattern
pattern = r"^[A-Za-z0-9\s-]+ - [A-Za-z]+, \d+$" # HL Mountain Frame - Silver, 46
paterrn_1 = r"^[A-Za-z0-9-]+ [A-Za-z]+, \d+$" # Touring-3000 Yellow, 44
filtered_product_df = product_df[product_df['Name'].str.match(pattern)]
filtered_product_df_1 = product_df[product_df['Name'].str.match(paterrn_1)]

In [228]:
# Split the string to extract required information (HL Mountain Frame - Silver, 46 -> HL Mountain Frame)
filtered_product_df['Group'] = filtered_product_df['Name'].str.rsplit(' ',n=3).str[0]
display(filtered_product_df[['ProductID', 'Group']].isnull().sum())
filtered_product_df = filtered_product_df[['ProductID', 'Group']]
filtered_product_df.head()

/tmp/ipykernel_33387/1170533106.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_product_df['Group'] = filtered_product_df['Name'].str.rsplit(' ',n=3).str[0]


ProductID    0
Group        0
dtype: int64

,ProductID,Group
209,680,HL Road Frame
210,706,HL Road Frame
221,717,HL Road Frame
222,718,HL Road Frame
223,719,HL Road Frame


In [229]:
# Split the string to extract required information (Touring-3000 Yellow, 44 -> Touring-3000)
filtered_product_df_1['Group'] = filtered_product_df_1['Name'].str.rsplit(' ',n=2).str[0]
display(filtered_product_df_1[['ProductID', 'Group']].isnull().sum())
filtered_product_df_1 = filtered_product_df_1[['ProductID', 'Group']]
filtered_product_df_1.head()

/tmp/ipykernel_33387/3965451237.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_product_df_1['Group'] = filtered_product_df_1['Name'].str.rsplit(' ',n=2).str[0]


ProductID    0
Group        0
dtype: int64

,ProductID,Group
253,749,Road-150
254,750,Road-150
255,751,Road-150
256,752,Road-150
257,753,Road-150


In [230]:
# Concatenate the two dataframes
concatenated_df = pd.concat([filtered_product_df, filtered_product_df_1], ignore_index=True)

new_data = pd.DataFrame([{'ProductID': 492, 'Group': 'Paint'},
                         {'ProductID': 493, 'Group': 'Paint'},
                         {'ProductID': 494, 'Group': 'Paint'},
                         {'ProductID': 495, 'Group': 'Paint'}])
concatenated_df = pd.concat([concatenated_df, new_data], ignore_index=True)

concatenated_df

,ProductID,Group
0,680,HL Road Frame
1,706,HL Road Frame
2,717,HL Road Frame
3,718,HL Road Frame
4,719,HL Road Frame
...,...,...
175,999,Road-750
176,492,Paint
177,493,Paint
178,494,Paint


In [231]:
# Inserting the cleaned data into the table
product_df = pd.merge(product_df, concatenated_df, on='ProductID', how='left')
product_df

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,...,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate,Group
0,1,Adjustable Race,AR-5381,False,False,,1000,750,0,0,...,,,,,2008-04-30 00:00:00,,,{694215B7-08F7-4C0D-ACB1-D734BA44C0C8},2014-02-08 10:01:36.827000000,NaN
1,2,Bearing Ball,BA-8327,False,False,,1000,750,0,0,...,,,,,2008-04-30 00:00:00,,,{58AE3C20-4F3A-4749-A7D4-D568806CC537},2014-02-08 10:01:36.827000000,NaN
2,3,BB Ball Bearing,BE-2349,True,False,,800,600,0,0,...,,,,,2008-04-30 00:00:00,,,{9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E},2014-02-08 10:01:36.827000000,NaN
3,4,Headset Ball Bearings,BE-2908,False,False,,800,600,0,0,...,,,,,2008-04-30 00:00:00,,,{ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B},2014-02-08 10:01:36.827000000,NaN
4,316,Blade,BL-2036,True,False,,800,600,0,0,...,,,,,2008-04-30 00:00:00,,,{E73E9750-603B-4131-89F5-3DD15ED5FF80},2014-02-08 10:01:36.827000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,995,ML Bottom Bracket,BB-8107,True,True,,500,375,44,101,...,M,,5,96,2013-05-30 00:00:00,,,{71AB847F-D091-42D6-B735-7B0C2D82FC84},2014-02-08 10:01:36.827000000,NaN
500,996,HL Bottom Bracket,BB-9108,True,True,,500,375,53,121,...,H,,5,97,2013-05-30 00:00:00,,,{230C47C5-08B2-4CE3-B706-69C0BDD62965},2014-02-08 10:01:36.827000000,NaN
501,997,"Road-750 Black, 44",BK-R19B-44,True,True,Black,100,75,343,539,...,L,U,2,31,2013-05-30 00:00:00,,,{44CE4802-409F-43AB-9B27-CA53421805BE},2014-02-08 10:01:36.827000000,Road-750
502,998,"Road-750 Black, 48",BK-R19B-48,True,True,Black,100,75,343,539,...,L,U,2,31,2013-05-30 00:00:00,,,{3DE9A212-1D49-40B6-B10A-F564D981DBDE},2014-02-08 10:01:36.827000000,Road-750


In [232]:
# Filling the NaN values with the original values
product_df['Group'] = product_df['Group'].fillna(product_df['Name'])

## Fill Standard Cost
After grouping, we need to calculate the standard cost for each of the group by using the average standard cost of each group.
Moreover, the purchased products will have standard cost equal to 0, since they are purchased. Therefore, we will use the average purchased price as the standard cost for them.

In [233]:
# Read the data from the database
purchase_order_df = pd.read_sql('SELECT * FROM purchaseorderdetail', conn)
purchase_order_df

/tmp/ipykernel_33387/868295767.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  purchase_order_df = pd.read_sql('SELECT * FROM purchaseorderdetail', conn)


,PurchaseOrderID,PurchaseOrderDetailID,DueDate,OrderQty,ProductID,UnitPrice,LineTotal,ReceivedQty,RejectedQty,StockedQty,ModifiedDate
0,1,1,2011-04-30,4,1,50.2600,201.0400,3,0,3,2011-04-23 00:00:00.000
1,2,2,2011-04-30,3,359,45.1200,135.3600,3,0,3,2011-04-23 00:00:00.000
2,2,3,2011-04-30,3,360,45.5805,136.7415,3,0,3,2011-04-23 00:00:00.000
3,3,4,2011-04-30,550,530,16.0860,8847.3000,550,0,550,2011-04-23 00:00:00.000
4,4,5,2011-04-30,3,4,57.0255,171.0765,2,1,1,2011-04-23 00:00:00.000
...,...,...,...,...,...,...,...,...,...,...,...
8840,4011,8841,2014-07-24,1000,880,20.5600,20560.0000,1000,0,1000,2015-08-12 12:25:46.470
8841,4012,8842,2014-07-24,6000,881,41.5700,249420.0000,6000,0,6000,2015-08-12 12:25:46.483
8842,4012,8843,2014-07-24,6000,882,41.5700,249420.0000,6000,0,6000,2015-08-12 12:25:46.483
8843,4012,8844,2014-07-24,6000,883,41.5700,249420.0000,6000,0,6000,2015-08-12 12:25:46.483


In [234]:
# Group the purchase order details by ProductID and calculate the average price
grouped_purchase_order_df = purchase_order_df.groupby('ProductID').agg({'LineTotal': 'sum', 'OrderQty': 'sum'}).reset_index()
grouped_purchase_order_df['AveragePrice'] = grouped_purchase_order_df['LineTotal'] / grouped_purchase_order_df['OrderQty']
grouped_purchase_order_df

,ProductID,LineTotal,OrderQty,AveragePrice
0,1,7.740565e+03,154,50.263409
1,2,6.287400e+03,150,41.916000
2,4,8.724901e+03,153,57.025500
3,317,1.246014e+06,44000,28.318500
4,318,1.532916e+06,44000,34.839000
...,...,...,...,...
260,939,2.390330e+06,49500,48.289500
261,940,1.455058e+06,23100,62.989504
262,941,1.766856e+06,28050,62.989504
263,948,2.277949e+06,27500,82.834504


In [235]:
# Merge the product dataframe with the grouped purchase order dataframe to get the new standard cost
product_df = pd.merge(product_df, grouped_purchase_order_df[['ProductID', 'AveragePrice']], on='ProductID', how='left')
product_df['AveragePrice'] = product_df['AveragePrice'].fillna(0)
product_df['StandardCost'] = [y if x == 0 else x for x, y in zip(product_df['StandardCost'], product_df['AveragePrice'])]
product_df = product_df.drop(columns=['AveragePrice'])
product_df

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,...,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate,Group
0,1,Adjustable Race,AR-5381,False,False,,1000,750,50.263409,0,...,,,,,2008-04-30 00:00:00,,,{694215B7-08F7-4C0D-ACB1-D734BA44C0C8},2014-02-08 10:01:36.827000000,Adjustable Race
1,2,Bearing Ball,BA-8327,False,False,,1000,750,41.916000,0,...,,,,,2008-04-30 00:00:00,,,{58AE3C20-4F3A-4749-A7D4-D568806CC537},2014-02-08 10:01:36.827000000,Bearing Ball
2,3,BB Ball Bearing,BE-2349,True,False,,800,600,0.000000,0,...,,,,,2008-04-30 00:00:00,,,{9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E},2014-02-08 10:01:36.827000000,BB Ball Bearing
3,4,Headset Ball Bearings,BE-2908,False,False,,800,600,57.025500,0,...,,,,,2008-04-30 00:00:00,,,{ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B},2014-02-08 10:01:36.827000000,Headset Ball Bearings
4,316,Blade,BL-2036,True,False,,800,600,0.000000,0,...,,,,,2008-04-30 00:00:00,,,{E73E9750-603B-4131-89F5-3DD15ED5FF80},2014-02-08 10:01:36.827000000,Blade
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,995,ML Bottom Bracket,BB-8107,True,True,,500,375,44.000000,101,...,M,,5,96,2013-05-30 00:00:00,,,{71AB847F-D091-42D6-B735-7B0C2D82FC84},2014-02-08 10:01:36.827000000,ML Bottom Bracket
500,996,HL Bottom Bracket,BB-9108,True,True,,500,375,53.000000,121,...,H,,5,97,2013-05-30 00:00:00,,,{230C47C5-08B2-4CE3-B706-69C0BDD62965},2014-02-08 10:01:36.827000000,HL Bottom Bracket
501,997,"Road-750 Black, 44",BK-R19B-44,True,True,Black,100,75,343.000000,539,...,L,U,2,31,2013-05-30 00:00:00,,,{44CE4802-409F-43AB-9B27-CA53421805BE},2014-02-08 10:01:36.827000000,Road-750
502,998,"Road-750 Black, 48",BK-R19B-48,True,True,Black,100,75,343.000000,539,...,L,U,2,31,2013-05-30 00:00:00,,,{3DE9A212-1D49-40B6-B10A-F564D981DBDE},2014-02-08 10:01:36.827000000,Road-750


In [236]:
# Calculate the average standard cost of each group
standard_cost_per_group = product_df.groupby('Group')['StandardCost'].mean().reset_index()
standard_cost_per_group.rename(columns={'StandardCost': 'GroupStandardCost'}, inplace=True)
standard_cost_per_group

,Group,GroupStandardCost
0,AWC Logo Cap,6.000000
1,Adjustable Race,50.263409
2,All-Purpose Bike Stand,59.000000
3,BB Ball Bearing,0.000000
4,Bearing Ball,41.916000
...,...,...
345,"Women's Mountain Shorts, M",26.000000
346,"Women's Mountain Shorts, S",26.000000
347,"Women's Tights, L",30.000000
348,"Women's Tights, M",30.000000


In [237]:
# Update the standard cost of each product based on the average standard cost of the group
product_df = pd.merge(product_df, standard_cost_per_group, on='Group', how='left')
product_df

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,...,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate,Group,GroupStandardCost
0,1,Adjustable Race,AR-5381,False,False,,1000,750,50.263409,0,...,,,,2008-04-30 00:00:00,,,{694215B7-08F7-4C0D-ACB1-D734BA44C0C8},2014-02-08 10:01:36.827000000,Adjustable Race,50.263409
1,2,Bearing Ball,BA-8327,False,False,,1000,750,41.916000,0,...,,,,2008-04-30 00:00:00,,,{58AE3C20-4F3A-4749-A7D4-D568806CC537},2014-02-08 10:01:36.827000000,Bearing Ball,41.916000
2,3,BB Ball Bearing,BE-2349,True,False,,800,600,0.000000,0,...,,,,2008-04-30 00:00:00,,,{9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E},2014-02-08 10:01:36.827000000,BB Ball Bearing,0.000000
3,4,Headset Ball Bearings,BE-2908,False,False,,800,600,57.025500,0,...,,,,2008-04-30 00:00:00,,,{ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B},2014-02-08 10:01:36.827000000,Headset Ball Bearings,57.025500
4,316,Blade,BL-2036,True,False,,800,600,0.000000,0,...,,,,2008-04-30 00:00:00,,,{E73E9750-603B-4131-89F5-3DD15ED5FF80},2014-02-08 10:01:36.827000000,Blade,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,995,ML Bottom Bracket,BB-8107,True,True,,500,375,44.000000,101,...,,5,96,2013-05-30 00:00:00,,,{71AB847F-D091-42D6-B735-7B0C2D82FC84},2014-02-08 10:01:36.827000000,ML Bottom Bracket,44.000000
500,996,HL Bottom Bracket,BB-9108,True,True,,500,375,53.000000,121,...,,5,97,2013-05-30 00:00:00,,,{230C47C5-08B2-4CE3-B706-69C0BDD62965},2014-02-08 10:01:36.827000000,HL Bottom Bracket,53.000000
501,997,"Road-750 Black, 44",BK-R19B-44,True,True,Black,100,75,343.000000,539,...,U,2,31,2013-05-30 00:00:00,,,{44CE4802-409F-43AB-9B27-CA53421805BE},2014-02-08 10:01:36.827000000,Road-750,343.000000
502,998,"Road-750 Black, 48",BK-R19B-48,True,True,Black,100,75,343.000000,539,...,U,2,31,2013-05-30 00:00:00,,,{3DE9A212-1D49-40B6-B10A-F564D981DBDE},2014-02-08 10:01:36.827000000,Road-750,343.000000


In [241]:
# Read material hierarchy table
material_hierarchy_df = pd.read_sql('SELECT * FROM materialhierachy', conn)
material_hierarchy_df

/tmp/ipykernel_33387/2307704911.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  material_hierarchy_df = pd.read_sql('SELECT * FROM materialhierachy', conn)


,ProductAssemblyID,ComponentID,ComponentName,TotalQuantity,StandardCost,ListPrice,BOMLevel,RecursionLevel
0,3,2,Bearing Ball,10.0,0.00,0.00,3,0
1,3,504,Cup-Shaped Race,2.0,0.00,0.00,3,0
2,3,461,Lock Ring,1.0,0.00,0.00,3,2
3,3,505,Cone-Shaped Race,2.0,0.00,0.00,3,2
4,316,486,Metal Sheet 5,1.0,0.00,0.00,4,0
...,...,...,...,...,...,...,...,...
2378,999,945,Front Derailleur,1.0,40.62,91.49,1,0
2379,999,948,Front Brakes,1.0,47.29,106.50,1,0
2380,999,949,LL Crankset,1.0,77.92,175.49,1,0
2381,999,952,Chain,1.0,8.99,20.24,1,0


In [ ]:
# Update the material hierarchy table
material_hierarchy_df = material_hierarchy_df.drop(columns=['StandardCost', 'ListPrice', 'ComponentName'])
material_hierarchy_df

,ProductAssemblyID,ComponentID,TotalQuantity,BOMLevel,RecursionLevel
0,3,2,10.0,3,0
1,3,504,2.0,3,0
2,3,461,1.0,3,2
3,3,505,2.0,3,2
4,316,486,1.0,4,0
...,...,...,...,...,...
2378,999,945,1.0,1,0
2379,999,948,1.0,1,0
2380,999,949,1.0,1,0
2381,999,952,1.0,1,0


## Write processed tables back to database

In [ ]:
from sqlalchemy import create_engine

host = 'materialserver-3163.j77.aws-ap-southeast-1.cockroachlabs.cloud'
port = '26257'
dbname = 'material'
user = 'vendor'
password = 'k_wEbjoSUNNXFzdjxdzxTg'
cluster_identifier = 'materialserver-3163'

username = 'your_username'
password = 'your_password'
host = 'your_cluster_id.your_region.cockroachlabs.cloud'
port = '26257'  # Default CockroachDB port
database = 'your_database'
cluster_identifier = 'your_cluster_id'

engine = create_engine(f'cockroachdb+psycopg2://{username}:{password}@{host}:{port}/{database}?sslmode=verify-full&options=--cluster={cluster_identifier}')